#### Imports and definitions

In [1]:
import os
from os.path import join
import sys
import json
import pickle
import numpy as np
import pandas as pd
#import cPickle
import gzip
from tensorflow.python.lib.io import file_io
from keras.layers import Dense, concatenate
from keras.models import Model, load_model
from sklearn.metrics import recall_score, precision_score, f1_score
from batch_generator import BatchGenerator, BatchSequence

/root/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def multiple_batch_generator(generator_num, **kwargs):
    """A generator to work with multiple inputs models
    
    We create a model with a list of multiple input layers when 
    we use :func:`keras.layers.concatenate`. However,
    :class:`batch_generator.BatchGenerator` returns a single tuple 
    with two arrays, which does not fit to a model with a multiple 
    input layers. Thus, with this generator, we create the necessary 
    input for such models.
    
    Arguments:
        generator_num {int} -- number of generators should be created
        \**kwargs -- See :class:`batch_generator.BatchGenerator`
        
    Yields:
        ([ndarray,...,ndarray], ndarray) -- in the tuple; list contains feature arrays from each generator, array out of the list contains the label set
    """
    #generators_list = [BatchGenerator(**kwargs, shuffle=False) for i in range(generator_num)]
    gen = BatchGenerator(**kwargs, shuffle=False)
    
    while True:
        nx = gen.next()
        Xy_list = [nx, nx, nx, nx]
        yield [Xy[0] for Xy in Xy_list], Xy_list[0][1]

#### Load models

In [3]:
model_vgg16 = load_model('models/VGG16-finetuned-1_epochs.h5')
for i, layer in enumerate(model_vgg16.layers):
    model_vgg16.layers[i].trainable = False
    model_vgg16.layers[i].name = '{}_{}'.format(layer.name, 'vgg16')
vgg16_out = model_vgg16.output

/root/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:270: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
model_vgg19 = load_model('models/VGG19-finetuned-4000_steps.h5')
for i, layer in enumerate(model_vgg19.layers):
    model_vgg19.layers[i].trainable = False
    model_vgg19.layers[i].name = '{}_{}'.format(layer.name, 'vgg19')
vgg19_out = model_vgg19.output

/root/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:270: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
model_xception = load_model('models/Xception-finetuned-4000_steps.h5')
for i, layer in enumerate(model_xception.layers):
    model_xception.layers[i].trainable = False
    model_xception.layers[i].name = '{}_{}'.format(layer.name, 'xception')
xception_out = model_xception.output

/root/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:270: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [6]:
model_resnet50 = load_model('models/ResNet50-finetuned-1_epochs.h5')
for i, layer in enumerate(model_resnet50.layers):
    model_resnet50.layers[i].trainable = False
    model_resnet50.layers[i].name = '{}_{}'.format(layer.name, 'resnet50')
resnet50_out = model_resnet50.output

/root/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:270: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


#### Concatenate

In [7]:
merge_0 = concatenate([vgg16_out, vgg19_out, xception_out, resnet50_out])
output = Dense(228, activation='sigmoid', name='main_output')(merge_0)
model = Model(inputs=[model_vgg16.input, model_vgg19.input, model_xception.input, model_resnet50.input], outputs=output)

#### Fit

In [3]:
model = load_model('models/stacked-300_steps.h5')

In [4]:
input_path = os.path.abspath('data/')

images_path_train = os.path.join(input_path, 'train/')
images_path_validation = os.path.join(input_path, 'validation/')
images_path_test = os.path.join(input_path, 'test/')

# Labels
with gzip.open('data/y_train.pickle','rb') as fp:
    y_train = pickle.load(fp)
with gzip.open('data/y_validation.pickle','rb') as fp:
    y_validation = pickle.load(fp)

In [ ]:
batch_size = 64
steps = int(len(y_train)/batch_size)
steps = 150
epochs = 1

model.compile(optimizer='adam', loss='binary_crossentropy')
input_num = 4#len(model.input_layers)

train_gen_multi = multiple_batch_generator(4, 
                                      input_dir=images_path_train,
                                      y=y_train,
                                      batch_size=batch_size)

for i in range(5):
    model.fit_generator(train_gen_multi, steps_per_epoch=steps, epochs=epochs)
    model.save( "models/stacked-{}_steps.h5".format((i+1)*steps+300) )

Epoch 1/1
150/150 [==============================] - 650s 4s/step - loss: 0.0743
Epoch 1/1
150/150 [==============================] - 628s 4s/step - loss: 0.0636
Epoch 1/1
150/150 [==============================] - 628s 4s/step - loss: 0.0605
Epoch 1/1
144/150 [===========================>..] - ETA: 25s - loss: 0.0591

#### Concatenate

In [ ]:
data = [[1,0,0,1,1],
        [0,0,1,0,1],
        [0,1,1,0,0],
        [1,1,1,1,1]]

In [ ]:
def to_categorical(labelmatrix):
    newlabels = []
    for labelrow in labelmatrix:
        labels = []
        for i,label in enumerate(labelrow):
            if label == 1:
                labels.append(i)
        newlabels.append(labels)
    return newlabels

In [ ]:
to_categorical(data)